# Implementation of Recurrent Neural Networks from Scratch
:label:`sec_rnn_scratch`

In this section we will implement an RNN
from scratch
for a character-level language model,
according to our descriptions
in :numref:`sec_rnn`.
Such a model
will be trained on H. G. Wells' *The Time Machine*.
As before, we start by reading the dataset first, which is introduced in :numref:`sec_language_model`.


In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.timemachine.TimeMachine
import jp.live.ugai.d2j.timemachine.Vocab
import jp.live.ugai.d2j.SeqDataLoader
import jp.live.ugai.d2j.util.StopWatch
import jp.live.ugai.d2j.util.Accumulator
import jp.live.ugai.d2j.util.Training
import kotlin.random.Random
import kotlin.Pair

// %load ../utils/djl-imports
// %load ../utils/plot-utils
// %load ../utils/Functions.java
// %load ../utils/PlotUtils.java

// %load ../utils/StopWatch.java
// %load ../utils/Accumulator.java
// %load ../utils/Animator.java
// %load ../utils/Training.java
// %load ../utils/timemachine/Vocab.java
// %load ../utils/timemachine/RNNModelScratch.java
// %load ../utils/timemachine/TimeMachine.java
// %load ../utils/timemachine/SeqDataLoader.java

In [2]:
val manager = NDManager.newBaseManager();

In [3]:
val batchSize = 32;
val numSteps = 35;
val timeMachine = SeqDataLoader.loadDataTimeMachine(batchSize, numSteps, false, 10000)
val trainIter = timeMachine.first
val vocab = timeMachine.second

## One-Hot Encoding

Recall that each token is represented as a numerical index in `trainIter`.
Feeding these indices directly to a neural network might make it hard to
learn.
We often represent each token as a more expressive feature vector.
The easiest representation is called *one-hot encoding*,
which is introduced
in :numref:`subsec_classification-problem`.

In a nutshell, we map each index to a different unit vector: assume that the number of different tokens in the vocabulary is $N$ (`vocab.length()`) and the token indices range from 0 to $N-1$.
If the index of a token is the integer $i$, then we create a vector of all 0s with a length of $N$ and set the element at position $i$ to 1.
This vector is the one-hot vector of the original token. The one-hot vectors with indices 0 and 2 are shown below.


In [4]:
manager.create(intArrayOf(0, 2)).oneHot(vocab.length())

ND: (2, 29) cpu() float32
[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., ... 9 more],
 [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., ... 9 more],
]


The shape of the minibatch that we sample each time is (batch size, number of time steps).
The `oneHot` function transforms such a minibatch into a three-dimensional NDArray with the last dimension equals to the vocabulary size (`vocab.length()`).
We often transpose the input so that we will obtain an
output of shape
(number of time steps, batch size, vocabulary size).
This will allow us
to more conveniently
loop through the outermost dimension
for updating hidden states of a minibatch,
time step by time step.


In [5]:
var X = manager.arange(10).reshape(Shape(2,5));
X.transpose().oneHot(28).getShape()

(5, 2, 28)

## Initializing the Model Parameters

Next, we initialize the model parameters for
the RNN model.
The number of hidden units `numHiddens` is a tunable hyperparameter.
When training language models,
the inputs and outputs are from the same vocabulary.
Hence, they have the same dimension,
which is equal to the vocabulary size.


In [6]:
fun getParams(vocabSize: Int, numHiddens: Int, device: Device): NDList {
    // Hidden layer parameters
    val W_xh: NDArray = normal(Shape(vocabSize.toLong(), numHiddens.toLong()), device)
    val W_hh: NDArray = normal(Shape(numHiddens.toLong(), numHiddens.toLong()), device)
    val b_h: NDArray = manager.zeros(Shape(numHiddens.toLong()), DataType.FLOAT32, device)
    // Output layer parameters
    val W_hq: NDArray = normal(Shape(numHiddens.toLong(), vocabSize.toLong()), device)
    val b_q: NDArray = manager.zeros(Shape(vocabSize.toLong()), DataType.FLOAT32, device)

    // Attach gradients
    val params = NDList(W_xh, W_hh, b_h, W_hq, b_q)
    for (param in params) {
        param.setRequiresGradient(true)
    }
    return params
}

fun normal(shape: Shape, device: Device): NDArray {
    return manager.randomNormal(0f, 0.01f, shape, DataType.FLOAT32, device)
}

## RNN Model

To define an RNN model,
we first need an `initRNNState` function
to return the hidden state at initialization.
It returns a NDArray filled with 0 and with a shape of (batch size, number of hidden units).


In [7]:
fun initRNNState(batchSize: Int, numHiddens: Int, device: Device): NDList {
    return NDList(manager.zeros(Shape(batchSize.toLong(), numHiddens.toLong()), DataType.FLOAT32, device))
}

The following `rnn` function defines how to compute the hidden state and output
at a time step.
Note that
the RNN model
loops through the outermost dimension of `inputs`
so that it updates hidden states `H` of a minibatch,
time step by time step.
Besides,
the activation function here uses the $\tanh$ function.
As
described in :numref:`sec_mlp`, the
mean value of the $\tanh$ function is 0, when the elements are uniformly
distributed over the real numbers.


In [8]:
fun rnn(inputs: NDArray, state: NDList, params: NDList): kotlin.Pair<NDArray, NDList> {
    // Shape of `inputs`: (`numSteps`, `batchSize`, `vocabSize`)
    val W_xh = params[0]
    val W_hh = params[1]
    val b_h = params[2]
    val W_hq = params[3]
    val b_q = params[4]
    var H = state[0]
    val outputs = NDList()
    // Shape of `X`: (`batchSize`, `vocabSize`)
    var X: NDArray
    var Y: NDArray
    for (i in 0 until inputs.size(0)) {
        X = inputs[i]
        H = X.dot(W_xh).add(H.dot(W_hh)).add(b_h).tanh()
        Y = H.dot(W_hq).add(b_q)
        outputs.add(Y)
    }
    return kotlin.Pair(if (outputs.size > 1) NDArrays.concat(outputs) else outputs[0], NDList(H))
}

With all the needed functions being defined,
next we create a class to wrap these functions and store parameters for an RNN model implemented from scratch.


In [9]:
/** An RNN Model implemented from scratch. */
class RNNModelScratch(
    var vocabSize: Int,
    var numHiddens: Int,
    device: Device,
    getParams: (Int, Int, Device) -> NDList,
    initRNNState: (Int, Int, Device) -> NDList,
    forwardFn: (NDArray, NDList, NDList) -> kotlin.Pair<NDArray, NDList>
) {
    var params: NDList
    var initState: (Int, Int, Device) -> NDList
    var forwardFn: (NDArray, NDList, NDList) -> kotlin.Pair<NDArray, NDList>

    init {
        params = getParams(vocabSize, numHiddens, device)
        initState = initRNNState
        this.forwardFn = forwardFn
    }

    fun forward(X: NDArray, state: NDList): kotlin.Pair<NDArray, NDList> {
        var X = X
        X = X.transpose().oneHot(vocabSize)
        return forwardFn(X, state, params)
    }

    fun beginState(batchSize: Int, device: Device): NDList {
        return initState(batchSize, numHiddens, device)
    }
}

Let us check whether the outputs have the correct shapes, e.g., to ensure that the dimensionality of the hidden state remains unchanged.


In [10]:
    val numHiddens = 512
    val getParamsFn = ::getParams
    val initRNNStateFn = ::initRNNState
    val rnnFn = ::rnn

    X = manager.arange(10).reshape(Shape(2, 5))
    val device = manager.device
    val net = RNNModelScratch(vocab.length(), numHiddens, device, getParamsFn, initRNNStateFn, rnnFn)
    val state = net.beginState(X.shape.shape[0].toInt(), device)
    val pairResult: kotlin.Pair<NDArray, NDList> = net.forward(X.toDevice(device, false), state)
    val Y: NDArray = pairResult.first
    val newState: NDList = pairResult.second
    println(Y.shape)
    println(newState[0].shape)

(10, 29)
(2, 512)


We can see that the output shape is (number of time steps $\times$ batch size, vocabulary size), while the hidden state shape remains the same, i.e., (batch size, number of hidden units).


## Prediction

Let us first define the prediction function
to generate new characters following
the user-provided `prefix`,
which is a string containing several characters.
When looping through these beginning characters in `prefix`,
we keep passing the hidden state
to the next time step without
generating any output.
This is called the *warm-up* period,
during which the model updates itself
(e.g., update the hidden state)
but does not make predictions.
After the warm-up period,
the hidden state is generally better than
its initialized value at the beginning.
So we generate the predicted characters and emit them.


In [11]:
/** Generate new characters following the `prefix`. */
fun predictCh8(prefix: String, numPreds: Int, net: RNNModelScratch, vocab: Vocab, device: Device): String {
    var state: NDList = net.beginState(1, device)
    val outputs: MutableList<Int> = ArrayList()
    outputs.add(vocab.getIdx("" + prefix[0]))
    val getInput = {
        manager.create(outputs[outputs.size - 1])
            .toDevice(device, false)
            .reshape(Shape(1, 1))
    }
    for (c in prefix.substring(1).toCharArray()) { // Warm-up period
        state = net.forward(getInput(), state).second
        outputs.add(vocab.getIdx("" + c))
    }
    var y: NDArray
    for (i in 0 until numPreds) {
        val pair = net.forward(getInput(), state)
        y = pair.first
        state = pair.second
        outputs.add(y.argMax(1).reshape(Shape(1)).getLong(0L).toInt())
    }
    val output = StringBuilder()
    for (i in outputs) {
        output.append(vocab.idxToToken[i])
    }
    return output.toString()
}

Now we can test the `predict_ch8` function.
We specify the prefix as `time traveller ` and have it generate 10 additional characters.
Given that we have not trained the network,
it will generate nonsensical predictions.


In [12]:
predictCh8("time traveller ", 10, net, vocab, manager.getDevice());

time traveller hkhkhkhkhk

## Gradient Clipping

For a sequence of length $T$,
we compute the gradients over these $T$ time steps in an iteration, which results in a chain of matrix-products with length  $\mathcal{O}(T)$ during backpropagation.
As mentioned in :numref:`sec_numerical_stability`, it might result in numerical instability, e.g., the gradients may either explode or vanish, when $T$ is large. Therefore, RNN models often need extra help to stabilize the training.

Generally speaking,
when solving an optimization problem,
we take update steps for the model parameter,
say in the vector form
$\mathbf{x}$,
in the direction of the negative gradient $\mathbf{g}$ on a minibatch.
For example,
with $\eta > 0$ as the learning rate,
in one iteration we update
$\mathbf{x}$
as $\mathbf{x} - \eta \mathbf{g}$.
Let us further assume that the objective function $f$
is well behaved, say, *Lipschitz continuous* with constant $L$.
That is to say,
for any $\mathbf{x}$ and $\mathbf{y}$ we have

$$|f(\mathbf{x}) - f(\mathbf{y})| \leq L \|\mathbf{x} - \mathbf{y}\|.$$

In this case we can safely assume that if we update the parameter vector by $\eta \mathbf{g}$, then

$$|f(\mathbf{x}) - f(\mathbf{x} - \eta\mathbf{g})| \leq L \eta\|\mathbf{g}\|,$$

which means that
we will not observe a change by more than $L \eta \|\mathbf{g}\|$. This is both a curse and a blessing.
On the curse side,
it limits the speed of making progress;
whereas on the blessing side,
it limits the extent to which things can go wrong if we move in the wrong direction.

Sometimes the gradients can be quite large and the optimization algorithm may fail to converge. We could address this by reducing the learning rate $\eta$. But what if we only *rarely* get large gradients? In this case such an approach may appear entirely unwarranted. One popular alternative is to clip the gradient $\mathbf{g}$ by projecting them back to a ball of a given radius, say $\theta$ via

$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$

By doing so we know that the gradient norm never exceeds $\theta$ and that the
updated gradient is entirely aligned with the original direction of $\mathbf{g}$.
It also has the desirable side-effect of limiting the influence any given
minibatch (and within it any given sample) can exert on the parameter vector. This
bestows a certain degree of robustness to the model. Gradient clipping provides
a quick fix to the gradient exploding. While it does not entirely solve the problem, it is one of the many techniques to alleviate it.

Below we define a function to clip the gradients of
a model that is implemented from scratch or a model constructed by the high-level APIs.
Also note that we compute the gradient norm over all the model parameters.


In [13]:
/** Clip the gradient. */
fun gradClipping(net: RNNModelScratch, theta: Int, manager: NDManager) {
    var result = 0.0
    for (p in net.params) {
        val gradient = p.gradient
        gradient.attach(manager)
        result += gradient.pow(2).sum().getFloat().toDouble()
    }
    val norm = Math.sqrt(result)
    if (norm > theta) {
        for (param in net.params) {
            val gradient = param.gradient
            gradient.muli(theta / norm)
        }
    }
}

## Training

Before training the model,
let us define a function to train the model in one epoch. It differs from how we train the model of :numref:`sec_softmax_scratch` in three places:

1. Different sampling methods for sequential data (random sampling and sequential partitioning) will result in differences in the initialization of hidden states.
1. We clip the gradients before updating the model parameters. This ensures that the model does not diverge even when gradients blow up at some point during the training process.
1. We use perplexity to evaluate the model. As discussed in :numref:`subsec_perplexity`, this ensures that sequences of different length are comparable.


Specifically,
when sequential partitioning is used, we initialize the hidden state only at the beginning of each epoch.
Since the $i^\mathrm{th}$ subsequence example  in the next minibatch is adjacent to the current $i^\mathrm{th}$ subsequence example,
the hidden state at the end of the current minibatch
will be
used to initialize
the hidden state at the beginning of the next minibatch.
In this way,
historical information of the sequence
stored in the hidden state
might flow over
adjacent subsequences within an epoch.
However, the computation of the hidden state
at any point depends on all the previous minibatches
in the same epoch,
which complicates the gradient computation.
To reduce computational cost,
we detach the gradient before processing any minibatch
so that the gradient computation of the hidden state
is always limited to
the time steps in one minibatch. 

When using the random sampling,
we need to re-initialize the hidden state for each iteration since each example is sampled with a random position.
Same as the `trainEpochCh3` function in :numref:`sec_softmax_scratch`,
`updater` is a general function
to update the model parameters.
It can be either the function implemented from scratch or the built-in optimization function in
a deep learning framework.


In [14]:
/** Train a model within one epoch. */
fun trainEpochCh8(
    net: RNNModelScratch,
    trainIter: List<NDList>,
    loss: Loss,
    updater: (Int, NDManager) -> Unit,
    device: Device,
    useRandomIter: Boolean
): Pair<Double, Double> {
    val watch = StopWatch()
    watch.start()
    val metric = Accumulator(2) // Sum of training loss, no. of tokens
    manager.newSubManager().use { childManager ->
        var state: NDList? = null
        for (pair in trainIter) {
            var X = pair[0].toDevice(device, true)
            X.attach(childManager)
            val Y = pair[1].toDevice(device, true)
            Y.attach(childManager)
            if (state == null || useRandomIter) {
                // Initialize `state` when either it is the first iteration or
                // using random sampling
                state = net.beginState(X.shape.shape[0].toInt(), device)
            } else {
                for (s in state) {
                    s.stopGradient()
                }
            }
            state.attach(childManager)
            var y = Y.transpose().reshape(Shape(-1))
            X = X.toDevice(device, false)
            y = y.toDevice(device, false)
            manager.engine.newGradientCollector().use { gc ->
                val pairResult = net.forward(X, state!!)
                val yHat: NDArray = pairResult.first
                state = pairResult.second
                val l = loss.evaluate(NDList(y), NDList(yHat)).mean()
                gc.backward(l)
                metric.add(floatArrayOf(l.getFloat() * y.size(), y.size().toFloat()))
            }
            gradClipping(net, 1, childManager)
            updater(1, childManager) // Since the `mean` function has been invoked
        }
    }
    return Pair(Math.exp((metric.get(0) / metric.get(1)).toDouble()), metric.get(1) / watch.stop())
}

The training function supports
an RNN model implemented
either from scratch
or using high-level APIs.


In [15]:
/** Train a model. */
fun trainCh8(
    net: RNNModelScratch,
    trainIter: List<NDList>,
    vocab: Vocab,
    lr: Int,
    numEpochs: Int,
    device: Device,
    useRandomIter: Boolean
) {
    val loss = SoftmaxCrossEntropyLoss()
//    val animator = Animator()
    // Initialize
    val updater = { batchSize: Int, subManager: NDManager ->
        Training.sgd(net.params, lr.toFloat(), batchSize, subManager)
    }
    val predict = { prefix: String -> predictCh8(prefix, 50, net, vocab, device) }
    // Train and predict
    var ppl = 0.0
    var speed = 0.0
    for (epoch in 0 until numEpochs) {
        val pair = trainEpochCh8(net, trainIter, loss, updater, device, useRandomIter)
        ppl = pair.first
        speed = pair.second
        if ((epoch + 1) % 10 == 0) {
//            animator.add(epoch + 1, ppl.toFloat(), "")
//            animator.show()
            println("${epoch + 1} : $ppl")
        }
    }
    println("perplexity: %.1f, %.1f tokens/sec on %s%n".format(ppl, speed, device.toString()))
    println(predict("time traveller"))
    println(predict("traveller"))
}

Now we can train the RNN model.
Since we only use 10000 tokens in the dataset, the model needs more epochs to converge better.


In [ ]:
    val numEpochs = Integer.getInteger("MAX_EPOCH", 500)

    val lr = 1
    trainCh8(net, trainIter, vocab, lr, numEpochs, manager.device, false)

10 : 13.6949028168055
20 : 10.718100385259126
30 : 9.644450712508839
40 : 9.078102043605819
50 : 8.577702529408375
60 : 8.302675927060575
70 : 7.983923919491217
80 : 7.762859344267908
90 : 7.5062167193031595
100 : 7.146752060146773
110 : 6.859298505800325
120 : 6.668297742885928
130 : 6.254953970250154
140 : 5.985675663791025
150 : 5.420180089105298
160 : 4.80670173492392
170 : 4.075168090049804
180 : 3.5090955926201337
190 : 2.9717029430213158
200 : 2.5226971135352505
210 : 2.1556108810147365
220 : 1.8650956788043624
230 : 1.6823604023827725
240 : 1.5312095284526932
250 : 1.4354322884433448
260 : 1.3413770443426507
270 : 1.2945296035951936
280 : 1.254750661530634
290 : 1.2060699658756628


Finally,
let us check the results of using the random sampling method.


In [ ]:
trainCh8(net, trainIter, vocab, lr, numEpochs, manager.getDevice(), true);

While implementing the above RNN model from scratch is instructive, it is not convenient.
In the next section we will see how to improve the RNN model,
such as how to make it easier to implement
and make it run faster.


## Summary

* We can train an RNN-based character-level language model to generate text following the user-provided text prefix.
* A simple RNN language model consists of input encoding, RNN modeling, and output generation.
* RNN models need state initialization for training, though random sampling and sequential partitioning use different ways.
* When using sequential partitioning, we need to detach the gradient to reduce computational cost.
* A warm-up period allows a model to update itself (e.g., obtain a better hidden state than its initialized value) before making any prediction.
* Gradient clipping prevents gradient explosion, but it cannot fix vanishing gradients.


## Exercises

1. Show that one-hot encoding is equivalent to picking a different embedding for each object.
1. Adjust the hyperparameters (e.g., number of epochs, number of hidden units, number of time steps in a minibatch, and learning rate) to improve the perplexity.
    * How low can you go?
    * Replace one-hot encoding with learnable embeddings. Does this lead to better performance?
    * How well will it work on other books by H. G. Wells, e.g., [*The War of the Worlds*](http://www.gutenberg.org/ebooks/36)?
1. Modify the prediction function such as to use sampling rather than picking the most likely next character.
    * What happens?
    * Bias the model towards more likely outputs, e.g., by sampling from $q(x_t \mid x_{t-1}, \ldots, x_1) \propto P(x_t \mid x_{t-1}, \ldots, x_1)^\alpha$ for $\alpha > 1$.
1. Run the code in this section without clipping the gradient. What happens?
1. Change sequential partitioning so that it does not separate hidden states from the computational graph. Does the running time change? How about the perplexity?
1. Replace the activation function used in this section with ReLU and repeat the experiments in this section. Do we still need gradient clipping? Why?
